In [79]:
import catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler#%%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import Pool, CatBoostRegressor


# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

Загрузим данные и сделаем первичный осмотр.

In [28]:
data = pd.read_csv('../datasets/autos.csv')
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Данные не в чистом виде, присутсвуют как и ошибки приведения типов, так и пропуски в данных, будем исправлять!

In [30]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Проанализируем конкретно в каких признаках у нас имеются пропуски.

In [31]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Первым делом исправим самую явную ошибку приведения типов - признак *Repaired*, его следует привести к типу Bool, предварительно заполним пропуски значением False, так как если поле пустой - скорее всего пользователи имели в виду, что она не чинилась, поэтому ничего и не вводили.

In [32]:
tmp_repaired = [True if i == 'yes' else False for i in data['Repaired']]

In [33]:
data['Repaired'] = tmp_repaired

Удалим признаки: *NumberOfPictures* - он содержит только нули и никакой информации не несёт, *DateCrawled*, *DateCreated*, *LastSeen*, *PostalCode* - признаки, никак не влияющие на цену.

In [34]:
data = data.drop(["DateCrawled","DateCreated","LastSeen","NumberOfPictures","PostalCode"], axis=1)

С остальными признаками чуть тяжелее - они все категориальные. Конечно, можно попытаться восстановить их на основе других данных, например восстановить тип техники по бренду и мощности двигателя, но такую группировку лучше согласовывать с заказчиком или спросить у более опытных коллег. Поэтому пока заполним пропуски "unknown".

In [35]:
data['VehicleType'] = data['VehicleType'].fillna('unknown')
data['Gearbox'] = data['Gearbox'].fillna('unknown')
data['Model'] = data['Model'].fillna('unknown')
data['FuelType'] = data['FuelType'].fillna('unknown')

## Обучение моделей

Для начала закодируем все категориальные переменные OneHotEncoding.

In [36]:
data_ohe = pd.get_dummies(data, drop_first=True)

Разделим на фичи и таргет.

In [37]:
features = data_ohe.drop('Price', axis=1)
target = data_ohe['Price']

Теперь поделим данные на обучающую и тестовую выборки в соотношении 3:1.

In [38]:
features_train_ohe, features_test_ohe, target_train_ohe, target_test_ohe = train_test_split(features, target, test_size=0.2, random_state=12345)

features_train_cat, features_test_cat, target_train_cat, target_test_cat = train_test_split(data.drop('Price', axis=1), data['Price'], test_size=0.2, random_state=12345)

Теперь приступим к обучению моделей, начнём с малого - Линейная Регрессия.

In [40]:
model_regression = LinearRegression()

model_regression.fit(features_train_ohe, target_train_ohe)
predictions = model_regression.predict(features_test_ohe)
print('RMSE:', mean_squared_error(y_true=target_test_ohe, y_pred=predictions, squared=False))

RMSE: 3207.000282114876


Теперь воспользуемся случайным лесом с подбором гипперпараметров.

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Лучшие параметры: {'max_depth': 40, 'n_estimators': 9}
Лучший счет: -1812.5861995571854

In [41]:
# model_forest = RandomForestRegressor(random_state=12345)
# params = {'max_depth': range (10, 41, 10), 'n_estimators': range(1, 11, 2)}
# grid = GridSearchCV(model_forest, params, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=10)
# grid.fit(features_train, target_train)
# print('Лучшие параметры:', grid.best_params_)
# print('Лучший счет:', grid.best_score_)

In [73]:
%%time
#**grid.best_params_
model_forest = RandomForestRegressor(random_state=42, n_estimators = 100, n_jobs=-1, criterion='mse')
model_forest.fit(features_train_ohe, target_train_ohe)

/Users/trupolog/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


CPU times: user 7min 54s, sys: 5.36 s, total: 8min
Wall time: 1min 3s


RandomForestRegressor(criterion='mse', n_jobs=-1, random_state=42)

/Users/trupolog/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


CPU times: user 7min 59s, sys: 5.64 s, total: 8min 5s
Wall time: 1min 2s


RandomForestRegressor(criterion='mse', n_jobs=-1, random_state=42)

In [74]:
%%time
predictions_forest = model_forest.predict(features_test_ohe)

CPU times: user 5.98 s, sys: 986 ms, total: 6.96 s
Wall time: 1.86 s


In [75]:
print('RMSE:', mean_squared_error(y_true=target_test_ohe, y_pred=predictions_forest, squared=False))

RMSE: 1747.859371944631


Теперь воспользуемся *Catboost*.


bestTest = 1725.410008
bestIteration = 999

CPU times: user 1h 29min 6s, sys: 6min 4s, total: 1h 35min 11s
Wall time: 17min 1s

In [48]:
# %%time
# catboost = CatBoostRegressor(loss_function='RMSE')
# parameters_cat = {'depth':[5,10], 'learning_rate':np.arange(0.1,1,0.2)}
# catboost_grid = catboost.grid_search(parameters_cat,
#                                      Pool(features_train_cat, target_train_cat, cat_features=["VehicleType","Gearbox",
#                                                                                       "Model","FuelType",
#                                                                                       "Brand","Repaired"]),
#                                      cv=3,
#                                      verbose=True,
#                                      plot=False)

0:	learn: 5809.8966985	test: 5855.3397921	best: 5855.3397921 (0)	total: 53.9ms	remaining: 53.8s
1:	learn: 5370.0408525	test: 5414.9732570	best: 5414.9732570 (1)	total: 167ms	remaining: 1m 23s
2:	learn: 4973.1573224	test: 5018.3430056	best: 5018.3430056 (2)	total: 203ms	remaining: 1m 7s
3:	learn: 4628.0113257	test: 4672.8383729	best: 4672.8383729 (3)	total: 234ms	remaining: 58.3s
4:	learn: 4327.3573281	test: 4372.0387052	best: 4372.0387052 (4)	total: 264ms	remaining: 52.6s
5:	learn: 4062.4474623	test: 4107.2284640	best: 4107.2284640 (5)	total: 302ms	remaining: 50s
6:	learn: 3831.7313195	test: 3875.7580524	best: 3875.7580524 (6)	total: 347ms	remaining: 49.3s
7:	learn: 3618.6418943	test: 3663.5216384	best: 3663.5216384 (7)	total: 383ms	remaining: 47.5s
8:	learn: 3432.2657002	test: 3478.4827459	best: 3478.4827459 (8)	total: 424ms	remaining: 46.7s
9:	learn: 3271.6961162	test: 3316.9203478	best: 3316.9203478 (9)	total: 475ms	remaining: 47.1s
10:	learn: 3132.7200039	test: 3177.7687803	best: 3

In [58]:
model_catboost = CatBoostRegressor(depth=catboost_grid["params"]["depth"],
                                    learning_rate=catboost_grid["params"]['learning_rate'], loss_function='RMSE')
model_catboost.fit(features_train_cat, target_train_cat, cat_features=["VehicleType","Gearbox",
                                                              "Model","FuelType",
                                                              "Brand","Repaired"])


0:	learn: 4198.5862380	total: 78.9ms	remaining: 1m 18s
1:	learn: 3921.4431721	total: 157ms	remaining: 1m 18s
2:	learn: 3677.0391653	total: 229ms	remaining: 1m 15s
3:	learn: 3462.3502408	total: 293ms	remaining: 1m 13s
4:	learn: 3279.6778561	total: 363ms	remaining: 1m 12s
5:	learn: 3113.4675537	total: 416ms	remaining: 1m 8s
6:	learn: 2972.3377961	total: 477ms	remaining: 1m 7s
7:	learn: 2843.1997154	total: 540ms	remaining: 1m 7s
8:	learn: 2730.8378431	total: 608ms	remaining: 1m 6s
9:	learn: 2631.6747930	total: 673ms	remaining: 1m 6s
10:	learn: 2546.4949107	total: 758ms	remaining: 1m 8s
11:	learn: 2472.5560550	total: 817ms	remaining: 1m 7s
12:	learn: 2408.9821175	total: 883ms	remaining: 1m 7s
13:	learn: 2347.4598970	total: 948ms	remaining: 1m 6s
14:	learn: 2295.3893735	total: 1.02s	remaining: 1m 7s
15:	learn: 2249.9695666	total: 1.07s	remaining: 1m 6s
16:	learn: 2211.5061082	total: 1.14s	remaining: 1m 5s
17:	learn: 2176.0554026	total: 1.2s	remaining: 1m 5s
18:	learn: 2145.8840243	total: 1.

In [97]:
train_dataset = Pool(features_train_cat, target_train_cat, cat_features=["VehicleType","Gearbox",
                                                                         "Model","FuelType",
                                                                         "Brand","Repaired"])
model = CatBoostRegressor(loss_function='RMSE')

bestTest = 1721.794207
bestIteration = 999

CPU times: user 6h 52min 26s, sys: 32min 19s, total: 7h 24min 45s
Wall time: 3h 10min 43s

In [98]:
%%time
grid = {'learning_rate': [0.1, 0.3],
        "max_depth": [2, 4, 6, 8, 10, 15],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)

21:	learn: 1640.8775709	test: 1836.1654088	best: 1836.1654088 (21)	total: 7.13s	remaining: 5m 16s
22:	learn: 1633.0463009	test: 1833.0633371	best: 1833.0633371 (22)	total: 7.45s	remaining: 5m 16s
23:	learn: 1623.5139268	test: 1828.9123911	best: 1828.9123911 (23)	total: 7.76s	remaining: 5m 15s
24:	learn: 1617.4671212	test: 1826.4666588	best: 1826.4666588 (24)	total: 8.09s	remaining: 5m 15s
25:	learn: 1601.2628202	test: 1821.2501491	best: 1821.2501491 (25)	total: 8.42s	remaining: 5m 15s
26:	learn: 1595.9798175	test: 1819.8175904	best: 1819.8175904 (26)	total: 8.73s	remaining: 5m 14s
27:	learn: 1590.5323266	test: 1818.4845267	best: 1818.4845267 (27)	total: 9.05s	remaining: 5m 14s
28:	learn: 1581.9144552	test: 1816.7089501	best: 1816.7089501 (28)	total: 9.39s	remaining: 5m 14s
29:	learn: 1575.0497489	test: 1814.3932530	best: 1814.3932530 (29)	total: 9.71s	remaining: 5m 14s
30:	learn: 1569.7254507	test: 1812.7413705	best: 1812.7413705 (30)	total: 10.1s	remaining: 5m 14s
31:	learn: 1560.3968

{'params': {'depth': 10, 'learning_rate': 0.1, 'l2_leaf_reg': 0.2},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
  

In [99]:
pred = model.predict(features_test_cat)
rmse = (np.sqrt(mean_squared_error(target_test_cat, pred)))
print('RMSE: {:.2f}'.format(rmse))

RMSE: 1696.79


{'loss_function': 'RMSE',
 'depth': 10,
 'learning_rate': 0.1,
 'l2_leaf_reg': 0.2}

In [103]:
model.get_params()

{'loss_function': 'RMSE',
 'depth': 10,
 'learning_rate': 0.1,
 'l2_leaf_reg': 0.2}

In [95]:
%%time

from lightgbm import LGBMRegressor

regressor = LGBMRegressor()
hyperparams = [{'num_leaves':[31, 100, 200],
                'learning_rate':[0.1, 0.3, 0.5],
                'random_state':[19072020]}]

print('# Tuning hyper-parameters for root_mean_squared_error')
print()
clf = GridSearchCV(regressor, hyperparams, scoring='neg_mean_squared_error', verbose=10)
clf.fit(features_train_ohe, target_train_ohe)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.6f for %r"% ((mean*-1)** 0.5, params))
print()

cv_RMSE_LGBMR = (max(means)*-1) ** 0.5

# Tuning hyper-parameters for root_mean_squared_error

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START learning_rate=0.1, num_leaves=31, random_state=19072020.....
[CV 1/5; 1/9] END learning_rate=0.1, num_leaves=31, random_state=19072020;, score=-3405491.236 total time=   9.4s
[CV 2/5; 1/9] START learning_rate=0.1, num_leaves=31, random_state=19072020.....
[CV 2/5; 1/9] END learning_rate=0.1, num_leaves=31, random_state=19072020;, score=-3441953.738 total time=  17.1s
[CV 3/5; 1/9] START learning_rate=0.1, num_leaves=31, random_state=19072020.....
[CV 3/5; 1/9] END learning_rate=0.1, num_leaves=31, random_state=19072020;, score=-3401497.028 total time=  14.0s
[CV 4/5; 1/9] START learning_rate=0.1, num_leaves=31, random_state=19072020.....
[CV 4/5; 1/9] END learning_rate=0.1, num_leaves=31, random_state=19072020;, score=-3410154.367 total time=  13.5s
[CV 5/5; 1/9] START learning_rate=0.1, num_leaves=31, random_state=19072020.....
[CV 5/5; 1/9] END learni

In [96]:
cv_RMSE_LGBMR

1717.5645537773837

In [77]:
%%time
predictions_cat = model_catboost.predict(features_test_cat)
print('RMSE:', mean_squared_error(y_true=target_test_cat, y_pred=predictions_cat, squared=False))

RMSE: 1703.1102254204443
CPU times: user 988 ms, sys: 158 ms, total: 1.15 s
Wall time: 273 ms


In [64]:
%%time

import lightgbm as lgb
lgb_train = lgb.Dataset(features_train_ohe, target_train_ohe)
lgb_test = lgb.Dataset(features_test_ohe, target_test_ohe, reference=lgb_train)
parameters_light = {'metric': 'l2', 'max_depth':10,"random_state": 42,"learning_rate":0.1}
light = lgb.train(parameters_light,
                  lgb_train,
                  num_boost_round=1000,
                  valid_sets=[lgb_train, lgb_test],
                  verbose_eval=100)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011728 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 967
[LightGBM] [Info] Number of data points in the train set: 283495, number of used features: 295
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 4414.416558
[100]	training's l2: 3.364e+06	valid_1's l2: 3.47816e+06
[200]	training's l2: 3.11148e+06	valid_1's l2: 3.26341e+06
[300]	training's l2: 2.9693e+06	valid_1's l2: 3.15841e+06
[400]	training's l2: 2.

In [76]:
%%time
print("LightGBM:",(mean_squared_error(target_test_ohe,light.predict(features_test_ohe),squared=False)))

LightGBM: 1719.0099735518481
CPU times: user 5.48 s, sys: 202 ms, total: 5.68 s
Wall time: 1.78 s


## Анализ моделей

Были обучены 4 модели: Линейная Регрессия, Случайный Лес, CatBoost и LightGBM. Начнем по порядку: наименьшую ошибку показал CatBoost - 1703, следом идёт LightGBM - 1719, затем Случайный Лес - 1747, а уже потом Линейная Регрессия - 3207. Разница между бустингом и регрессией колосальна. А вот разница между тремя первыми местами не сильно заметна, особенно если рассматривать только бустинги - всего в 16 евро.

Критерии, которые важны заказчику:
-качество предсказания;
-время обучения модели;
-время предсказания модели.


Отбросим линейную регрессии в силу ее слабой RMSE и за каждый критерий будем давать от 0 до 2 баллов за позицию в критериях.
Качество предсказания: CatBoost - 2 балла, LightGBM - 1 балл, Случайный Лес - 0 баллов.
Время обучения модели: CatBoost - 0 баллов, LightGBM - 2 балл, Случайный Лес - 1 балла.
Время предсказания модели: CatBoost - 2 балла, LightGBM - 1 балл, Случайный Лес - 0 баллов.

Таким образом у CatBoost - 4 балла, у LightGBM - 4 балла и у Случайного Лесса 1 балл.
Но в данном случае выбор падет на LightGBM, так как уж намного быстрее эта модель обучилась в отличие от CatBoost.

## Вывод

Была проведена работа по обработке данных, обучению модель и анализа лучшей. В самом начале мы первично изучали данные и начали их обрабатывать - было много пропусков и не все признаки соответствовали своим типам. После разбили данные на обучающую и тестовую выборки несколько раз, так как разные модели могут принимать на вход только определенные данные - некоторые уже закодированные, а некоторые могут принимать и не закодированные данные. Обучив и проверив модели на тестовых выборках - стало ясно, что линейная регрессия такая себе модель для данной задачи, а после сравнительного анализа стало ясно, что лучшая модель в нашем случае - LightGBM!

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей